In [10]:
#Calculate how different two sequences are from each other. The bigger the number, the more different
#p and q are both sequence strings that should be the same length
#Return an integer
def HammingDistance(p, q):
    dist = 0
    if len(p)==len(q): #check to make sure sequences are the same length
        for i in range(len(p)):
            if p[i]==q[i]: #if the sequence has the same letter at the same position, don't add to score
                dist+=0
            else:
                dist+=1
    return dist

In [11]:
def Neighbors(pattern, d):
    if d == 0: #if you're allowed no mismatches, just return the pattern
        return pattern
    elif len(pattern) == 1: #if the pattern is only 1nt long, return all the possible nucleotides
        return {'A', 'C', 'G', 'T'}
    neighborhood = []
    suffix_neighbors = Neighbors(pattern[1:], d) #recursion. Logically goes off the rails for me here.
    for text in suffix_neighbors:
        nucleotides = {'A', 'C', 'G', 'T'}
        if HammingDistance(pattern[1:], text) < d:
            for x in nucleotides:
                neighborhood.append(x + text)
        else:
            neighborhood.append(pattern[0] + text)
    return neighborhood

In [12]:
def MotifEnumeration(dna, k, d):
    motifs = []
    for i in range(len(dna)): #cycle through the dna list
        if i == 0: #for the first string
            for j in range(len(dna[i])-k+1): #cycle through the first string
                pattern = (dna[i][j:j+k]) #identify a sequence of k-length in the first string
                pattern_p=Neighbors(pattern, d) #use that pattern to identify all possibilities with d mismatches
                if isinstance(pattern_p, str)==True:  #If there's only one value, it's a string not a list.
                    motifs.append(pattern_p) #add to potential motif list
                else:
                    motifs.extend(pattern_p) #add to potential motif list
    motifs = list(set(motifs)) #get rid of duplicates
    temp = {}
    removemotif = []
    for j in range(1, len(dna)): #now look at the other dna strings
        for motif in motifs: #cycle through the motifs and see if they're in the other dna strings
            #print(motif) #checking that things are working
            for i in range(len(dna[j])-k+1):
                if HammingDistance(motif, dna[j][i:i+k]) <= d: #identify if motif occurs in the string
                    if motif in list(temp.keys()):
                        temp[motif]+=1
                    else:
                        temp[motif]=1
                else:
                    if motif not in list(temp.keys()):
                        temp[motif]=0
            if temp[motif] == 0: #identify motifs that should be removed because they don't occur in the string
                removemotif.append(motif)
        #print(motifs) #checking that things are working
        temp = {} #reset temp before cycling to the next dna string
    for motif in list(set(removemotif)): #remove all the motifs not in every string
        motifs.remove(motif)
    return motifs

text = ["CTTTGCTCGAAACCCTGTGGACTTC", "CATGGTTAATTACCTAAACCCGTTC", "CTATTGTGTCAAACCCCCGGGTGCG", "ACGGGTGTGCACCGCCAACTCATCC", 
        "AATGATCGTAGATCCGAGCCGCATT", "TGCTACACCCGGATTCGTGAAGGGA"]
a = MotifEnumeration(text, 5, 2)
print(*a)

GCCGA GTGGT CTACG GATGC TCCTC CATAA TCCAC TACGC GGATA CGGGG TGAAG TACCT GTATC CGCGG CTCAA GGCGT CCAGC CTTTG GCATT CGGTC ACTAA CGACC ACCTG TACGG GGCGA AGTGG ACGTG TGTAC GTAGT ACTCC CAGCT CATCC CTGGC CCCCT CGCTC GTGCC CCCTC ACCCG CGTTG CCCGA GCTCG GGGTT GTACC TAAGC GGTCT ACTCA GGAAC CTGGG TGTTT TTATT CGTAA ACCAC CCTAA CTGAC AAGCT ATACT GTCGA CCGAG CGTAT CTGAA AACCT ACCGA AACGT ACCTT ATTGC AACGC ATTCT GACGC GTAAG CTGCT GTGTA ACCGG CTGGA CACCT CGTGG GAACA TTTAT CCTGG AGAGC CAATC GTGAC AACCA CCCGT AGTGT GTGGG AGCGT ATATC TATGC GATCA CATCG AGCAC AAGTC CGTCT CCTCC CTTAA CGGCC AAATG GCTTC TACTT CGGTA CTCAT GTCGT CCACT AAATC GTGCA TCTAA GAATC TCCGG TCCGT GCCCT CTTGC TGGCT ACGTC CACGC CCAGA TCACA AACAA CGTCG CAAGA TGCCT ACATC CATAC AATGT TTCTC GGCCG ACATT CGAGA ATGCA GCTGA CACAA CTCCG CCCAA ACACG AGTTG TCCCG ATCGG GGATG GGGTC GATTC TTACC ACTTC CGTAC CGATT TCACC AACCG CTCGT CCCCC CAGTG GCCTC CTCCT TATGT TTTGA GGTGC GTGAG TTGAT TGCAT CGAAA AGTTT AATTT TCCGA CCCCA GCGGT AGGTC AAAGG TGGCA AGCCA TTGG

In [13]:
#Count how frequently a symbol occcurs at a certain position when given a list of motifs of the same length
#motifs is a list of strings of the same length
#count output is a dictionary with A C G T as the keys
def Count(motifs):
    count = {} # initializing the count dictionary
    k = len(motifs[0]) #get the length of each motif row (x axis)
    for symbol in "ACGT": #Make A C G T a key
        count[symbol] = [] #make an empty list for each key
        for j in range(k): #for the length of the motif...
            count[symbol].append(0) #...for the key, put a zero for each position in the list for now
    t = len(motifs) #get the number of keys in the dictionary (y axis)
    for i in range(t): #for every key...
        for j in range(k): #for list position in each key...
            symbol = motifs[i][j] #for this specific position for a key...
            count[symbol][j] += 1 #Add one every time this position occurs in the matrix at this position
    return count

In [14]:
#similar to Count, but get a ratio instead. All columns should add to 1.
#motifs input is a list of strings of the same length
#profile output is a dictionary with A C G T as keys
def Profile(motifs):
    t = len(motifs)
    k = len(motifs[0])
    profile = Count(motifs)
    sub= Count(motifs)
    for x in range(t): #for key in position x...
        for y in range(k): #for list position in that key...
            symbol=motifs[x][y]
            profile[symbol][y]=(sub[symbol][y])/t
    return profile

motifs = [
"TCGGGGGTTTTT",
"CCGGTGACTTAC",
"ACGGGGATTTTC",
"TTGGGGACTTTT",
"AAGGGGACTTCC",
"TTGGGGACTTCC",
"TCGGGGATTCAT",
"TCGGGGATTCCT",
"TAGGGGAACTAC",
"TCGGGTATAACC"
]

Profile(motifs)

#Need this for calculating log2 values
import math

#Calculate the randomness of a motif
#uses a list of dna strings as the input
#output is a number standing for the entropy
#This wasn't a very useful function.
def entropy(motifs):
    prof = Profile(motifs)
    #result = []
    result2 = 0
    for i in range(len(prof["A"])):
        temp = 0
        for key in prof:
            if prof[key][i] != 0: #don't want to take the log of zero values
                temp += abs(prof[key][i]*math.log2(prof[key][i]))
            else:
                temp += 0
        #result.append(temp) #this is really just here to see that things were working
        result2 += temp
    return result2

0.5*math.log2(0.5)*2

0.25*math.log2(0.25)*4

1*math.log2(1)

0.25*math.log2(0.25)*2+0.5*math.log2(0.5)

entropy(motifs)

9.916290005356972

In [15]:
#Identify the motif with the lowest score
#pattern is a string
#dna is a list of strings
#return a list with the best match for each dna string 
def Motifs(pattern, dna):
    motifs = []
    k= len(pattern)
    if isinstance(dna, str)==True: #verify if the input is a list or a string
        new_dna = []
        new_dna.append(dna)
        dna = new_dna #If a string, make it a list so it works with the rest of the function
    for seq in dna:
        score = k
        temp_motif = {}
        for i in range(len(seq)-k+1): #find the motifs with the lowest score
            new_score = HammingDistance(pattern, seq[i:i+k])
            if new_score < score:
                temp_motif[seq[i:i+k]]=new_score
        if len(temp_motif)==0: #if you get nothing lower, just return the first k-mer
            motifs.append(seq[0:k])
        else:
            low_score = min(temp_motif.values()) #Find the lowest score
            for key in temp_motif: #identify the motif with the lowest score
                if temp_motif[key] == low_score:
                    motifs.append(key)
                    break #just want one motif per dna string, so end search once you find it
    return motifs

dna = ["TTACCTTAAC", "GATATCTGTC", "ACGGCGTTCG", "CCCTAAAGAG"]    

Motifs("AAA", dna)

['TAA', 'ATA', 'ACG', 'AAA']

In [16]:
#Return a score for how different a pattern is from the dna input
#pattern is a string
#dna is a list of strings
#output score is an integer
def d(pattern, dna):
    motifs = Motifs(pattern, dna)
    if len(motifs)==0:
        return len(pattern)
    score = 0
    for motif in motifs:
        x = HammingDistance(pattern, motif)
        score += x
    return score

d("AAA", dna)

4

In [17]:
#Search for the k-mer that is in at least one string and has the lowest score for all strings
#dna can be either a string or a list of strings.
#k is an integer for how long the output is supposed to be
#output is median, which is a string
def MedianString(dna, k):
    kmers = []
    median = ""
    if isinstance(dna, str)==True: #If dna is a string, make it a list
        new_dna = []
        new_dna.append(dna)
        dna = new_dna 
    distance = len(dna[0])*len(dna)
    for seq in dna: #for each string in the dna list
        for i in range(len(seq)-k+1): #make a list of all kmers
            kmers.append(seq[i:i+k])
    kmers = list(set(kmers)) #remove duplicates
    for kmer in kmers: 
        if distance > d(kmer, dna): #if the differences are less than the total length
            distance = d(kmer, dna) #update the d output as the new min
            median = kmer #return the kmer with the lowest distance
        elif distance == d(kmer, dna):
            median = kmer
    return median

dna = ["AAATTGACGCAT", "GACGACCACGTT", "CGTCAGCGCCTG", "GCTGAGCACCGG", "AGTTCGGGACAG"]
dna2 = ["TGATGATAACGTGACGGGACTCAGCGGCGATGAAGGATGAGT", "CAGCGACAGACAATTTCAATAATATCCGCGGTAAGCGGCGTA", 
        "TGCAGAGGTTGGTAACGCCGGCGACTCGGAGAGCTTTTCGCT", "TTTGTCATGAACTCAGATACCATAGAGCACCGGCGAGACTCA", 
        "ACTGGGACTTCACATTAGGTTGAACCGCGAGCCAGGTGGGTG", "TTGCGGACGGGATACTCAATAACTAAGGTAGTTCAGCTGCGA", 
        "TGGGAGGACACACATTTTCTTACCTCTTCCCAGCGAGATGGC", "GAAAAAACCTATAAAGTCCACTCTTTGCGGCGGCGAGCCATA", 
        "CCACGTCCGTTACTCCGTCGCCGTCAGCGATAATGGGATGAG", "CCAAAGCTGCGAAATAACCATACTCTGCTCAGGAGCCCGATG"]
dna3 = ["GGCGGGAGGCAAGAGTCCGCGTCTCAAGATTATAAGCGGGGG", "GCGCCACACGAAGTTGGATACCCTTCGAGTGAGGGAAGACAA", 
        "TGTTTAAGGCAACCTTTTGCTTCTCAACCACGCGCTGGCTCC", "CCTATCGGACCCCGCAATGACTCAAGCCAAGTTGCGAGGGTT", 
        "ATAATATCGTTCTTAGATAATCGCACCCTCAGAGTAAGACAA", "CTCACTAGACAACAACCTAGATGCTATCTGTGGGTTACTAAG", 
        "TGCAGTGTCACAGACCGAGAGGCTCGCTCGAGCCAAAGCACA", "GAGTGGTCTCGACACCTCAGTCAAACCGCATCCTGAGCCCGG", 
        "TTGCATTCATAGGCTATCTCCTGGTTGTTACCCTACAGCCAA", "AGTCAATAGCCTCTACCGCATATAATCCAGCAGTAGGAGCGA"]
MedianString(dna, 3)
MedianString(dna2, 6)
MedianString(dna3, 6)

'AGACAA'

In [18]:
dna4 = ["CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC", "GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC", "GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG"]
MedianString(dna4, 7)

'TAGTTTC'

In [19]:
#Similar to previous, but search kmers not in the main text (uses Neighbors)
#output is now a list of strings
def MedianString2(dna, k):
    kmers = []
    median = []
    if isinstance(dna, str)==True: #If dna is a string, make it a list
        new_dna = []
        new_dna.append(dna)
        dna = new_dna 
    distance = len(dna[0])*len(dna) #technically, an entry could poorly match everything
    for seq in dna:
        for i in range(len(seq)-k+1):
            kmers.append(seq[i:i+k]) #make a list of all kmers in the first dna string
    kmers = list(set(kmers)) #remove duplicates
    kmers2 = []
    for i in range(len(kmers)):#add more kmers that are not just in the main text
        kmers2.append(kmers[i])
        newkmers = Neighbors(kmers[i], k%2+1) #arbitrarily decided on this since no number of mismatches is stated. Essentially, adding things that can over half mismatches.
        kmers2.extend(newkmers) #add new kmers to the list
    kmers2 = list(set(kmers2)) #remove duplicates
    for kmer in kmers2:
        if distance > d(kmer, dna): #if the new kmer has a small distance...
            distance = d(kmer, dna) #reset the distance to the new min
            median = [] #reset the median to get rid of anything that performed worse
            median.append(kmer)
        elif distance == d(kmer, dna): #if distance is the same, add it to the list so we know there are multiple motifs that perform equally well
            median.append(kmer)
    return median

test = ["ATA","ACA","AGA","AAT","AAC"]

MedianString2(test, 3)

MedianString2(dna, 3)

dna = ['TGTTGGAAATATCACGTGGACCCTCTCGTGCAGTGGATATAA',
 'TGTCCGCGGCAACACCCTAGTGCCGGAGCCGGATCTGTGCAA',
 'TGGGCGTGGACGTACCCTGACCCCCAATTTCGTGAGTATCAA',
 'CTTCATAATTATAACCCTATTTTCAGTCACGAGCACCCTAAG',
 'TACCCTCCAATACAGGACGTACCTAGGTGCGCGCGCCAGACG',
 'TGTATTAACCAAAGATAGTCCCGGTACCCTGCGCGCTCGGCA',
 'AACCCTCCACTCCGGAATAATGTTTACATACCCCCTTTAATT',
 'CTGCTAGCATAGAGCATTGCTTGTTTGGCTGCCCTACACCCT',
 'GTGCAGGGCGTAAGATAGAACCCTGCTACGAGCCGTGGAAAC',
 'CAGAACACCCGCGCAAGAGACCCTTCGGTAGGAATCATGCCT']

MedianString(dna, 6)
MedianString2(dna, 6)

['CACCCT', 'TACCCT', 'AACCCT']

In [22]:
MedianString2(dna4, 7)

['TAGTTTC', 'GAACCAC', 'AATCCTA', 'GTAGGAA']

In [23]:
#Find the likelihood of a certain sequence occuring within a profile of dna strings (probability of each nucleotide at each position)
#text is a dna string
#profile is a dictionary with keys A, C, G, T and the same number of entries as text
#output is an integer
def Pr(text, profile):
    p=1
    for i in range(len(text)):
        p=p*profile[text[i]][i]
    return p

profile = {

    'A': [0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.9, 0.1, 0.1, 0.1, 0.3, 0.0],
    'C': [0.1, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.1, 0.2, 0.4, 0.6],
    'G': [0.0, 0.0, 1.0, 1.0, 0.9, 0.9, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    'T': [0.7, 0.2, 0.0, 0.0, 0.1, 0.1, 0.0, 0.5, 0.8, 0.7, 0.3, 0.4]
}

Pr("TCGTGGATTTCC", profile)

0.0

In [24]:
profile = {

    'A': [0.4, 0.3, 0.0, 0.1, 0.0, 0.9],
    'C': [0.2, 0.3, 0.0, 0.4, 0.0, 0.1],
    'G': [0.1, 0.3, 1.0, 0.1, 0.5, 0.0],
    'T': [0.3, 0.1, 0.0, 0.4, 0.5, 0.0]
}

Pr("GAGCTA", profile)

0.0054